## imports

In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2025-04-11 20:43:08.250347: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-11 20:43:08.357765: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 20:43:13.812569: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [15]:
from plotly import express as px
import os
import numpy as np
import pickle
from copy import deepcopy

## data load

In [3]:
data_path = "/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto/folds_training_dicts.pkl"

In [5]:
with open(data_path, "rb") as f:
    folds_training_dicts = pickle.load(f)

2025-04-11 20:44:33.093863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## fetch wrongly negative uniprots

In [9]:
fold = folds_training_dicts[0]
fold.keys()

dict_keys(['sizes_train', 'components_train', 'num_patches_train', 'uniprots_train', 'labels_train', 'sizes_validation', 'components_validation', 'num_patches_validation', 'uniprots_validation', 'labels_validation', 'sizes_test', 'components_test', 'num_patches_test', 'uniprots_test', 'labels_test'])

In [10]:
GT_positive_uniprots = []
for set_name in ['DUB', 'E1', 'E2', 'E3', 'ubiquitinBinding']:
    GT_positive_uniprots.extend([x[:-4] for x in os.listdir(f'/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/{set_name}') if 'pdb' in x])
GT_positive_uniprots = set(GT_positive_uniprots)
len(GT_positive_uniprots)

78433

In [12]:
negative_uniprots = []
for set_name in ['train', 'validation', 'test']:
    for i in np.where(fold[f'labels_{set_name}'] == 0)[0]:
        negative_uniprots.append(fold[f'uniprots_{set_name}'][i])
negative_uniprots = set(negative_uniprots)
len(negative_uniprots)

76483

In [13]:
wrongly_negative_uniprots = negative_uniprots.intersection(GT_positive_uniprots)
len(wrongly_negative_uniprots)

547

## create curated dataset

In [41]:
curated_folds_training_dicts = deepcopy(folds_training_dicts)

In [42]:
for curated_fold in curated_folds_training_dicts:
    for set_name in ['train', 'validation', 'test']:
        curated_set_labels = np.array([1 if x in GT_positive_uniprots else 0 for x in curated_fold[f'uniprots_{set_name}']]) 
        curated_fold[f'labels_{set_name}'] = tf.convert_to_tensor(curated_set_labels)

In [43]:
with open('curated_folds_training_dicts.pkl', 'wb') as f:
    pickle.dump(curated_folds_training_dicts, f)

In [44]:
curated_fold = curated_folds_training_dicts[0]

In [45]:
changed_count = 0
for set_name in ['train', 'validation', 'test']:
    changed_count += (np.sum(curated_fold[f'labels_{set_name}']) - np.sum(fold[f'labels_{set_name}']))
assert changed_count == len(wrongly_negative_uniprots)

In [48]:
assert type(curated_fold['labels_train']) == type(fold['labels_train'])